In [1]:
import json
import ijson
from tqdm import tqdm
from indexingcode.utils.Preprocessor import Preprocessor
import gensim
import nltk
import xml.etree.ElementTree as ET

In [16]:
def get_text(node):
    try:
        return node.text
    except AttributeError:
        return None

In [27]:
preprocessor = Preprocessor(gensim.utils.tokenize, nltk.PorterStemmer())

[nltk_data] Downloading package stopwords to /home/wojtek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
et = ET.parse('/media/wojtek/SanDisk/desc2022.xml')

In [24]:
# load meshes XML
text_to_tree = {}
root = et.getroot()
omit_count = 0
with tqdm(root.findall('DescriptorRecord')) as bar:
    for record in bar:
        tree_text = get_text(record.find('TreeNumberList/TreeNumber'))
        name = get_text(record.find('DescriptorName/String'))
        if name is None or tree_text is None:
            omit_count += 1
            bar.set_description(str(omit_count))
            continue
        # first_tree = tree_text.split('.')[0]
        first_tree = tree_text[0]
        text_to_tree[name] = first_tree

2: 100%|██████████| 30194/30194 [00:00<00:00, 223325.21it/s]


In [30]:
with open('out/artprep/meshes.csv', encoding='ISO-8859-1') as meshes_file:
    with open('/media/wojtek/SanDisk/BioAsq2021/allMeSH_2021.json', encoding='ISO-8859-1') as bioasq_file:
        with open('out/artprep/meshes_tree.csv', 'w') as meshes_tree_write_file:
            num_to_text = {}
            for article in tqdm(ijson.items(bioasq_file, 'articles.item'), total=15559157, desc='Num to text dictionary creation'):
                for num, text in zip(preprocessor.preprocess_mesh(article['meshMajor']), article['meshMajor']):
                    if num not in num_to_text:
                        num_to_text[num] = text

            num_to_tree = {}
            omit_count = 0
            for num, text in tqdm(num_to_text.items(), desc='Num to tree transformation'):
                if text in text_to_tree:
                    first_tree = text_to_tree[text]
                    num_to_tree[num] = first_tree
                else:
                    omit_count += 1

            for line in tqdm(meshes_file, total=15559157, desc='First tree writing'):
                nums = list(map(int, line.split(',')))
                trees = list(filter(lambda x: x is not None, [num_to_tree[n] if n in num_to_tree else None for n in nums]))
                most_freq_tree = max(set(trees), key=trees.count)
                meshes_tree_write_file.write(most_freq_tree + '\n')


First tree writing: 100%|██████████| 15559157/15559157 [01:27<00:00, 177167.79it/s]
